## Синтаксический анализатор SpaCy

Одним из синтаксических анализаторов является Spacy. Библиотеки можно поставить при помощи pip, а как добавить **языковые модели** можно посмотреть [здесь](https://spacy.io/usage/models).

In [28]:
# Импортируемм нужные библиотеки.
import spacy
from spacy import displacy
from pprint import pprint
from collections import defaultdict
from tqdm import tqdm

In [29]:
# Загружаем языковую модель.
nlp = spacy.load("ru_core_news_sm")

Для анализа используем тексты новостей с сайта Лента.ру.

In [31]:
with open("data/lenta2018.txt", encoding="utf-8") as news_file: # Файл с новостями.
    text_news = [n.split("-----\n")[1] for n in news_file.read().split("=====\n")[1:]]
    

In [32]:
text_news[11]

'Ракета-носитель «Союз-2.1а» с 11 спутниками стартовала с космодрома Восточный. Об этом сообщает Космический центр «Южный», который ведет прямую трансляцию запуска.Корабль должен доставить на орбиты 11 спутников, два из которых — российские спутники дистанционного зондирования Земли «Канопус-В» №3 и №4.Это уже третий пуск с Восточного. Предыдущий состоялся 28 ноября 2017 года и завершился неудачей — из-за неполадок с разгонным блоком была утеряна головная часть ракеты «Союз-2.1б» (с 19 спутниками на борту). В госкорпорации «Роскосмос» объяснили инцидент ошибкой в программном обеспечении разгонного блока и ракеты-носителя, заложенной еще 20 лет назад. Позже вице-премьер России Дмитрий Рогозин заявил, что при подготовке запуска сотрудники перепутали космодромы: «Союз-2.1б» запускали с Восточного, при этом параметры траектории рассчитали с учетом старта с Байконура.В ближайшие годы «Роскосмос» ставит задачу повысить долю успешных запусков ракет-носителей до 99 процентов.\n'

Дерево в Spacy представляется в виде дерева зависимостей: как совокупность вершин и связей между ними. Каждая вершина соответствует слову из предложения, у слова есть лемма (поле `lemma_`) и токен (поле `text`).

In [33]:
# Передаем текст новости в Spacy для разбора, получаем объект документа с результатами разбора.
doc = nlp(text_news[11])
# Перебираем предложения в тексте. 
# Но так как мне нужны первые два предложения, я превращаю итерируемый объект sents в список.
for s in list(doc.sents)[:2]:
    pprint(list([f'{t.lemma_} ({t.text})' for t in s]))

['ракета (Ракета)',
 '- (-)',
 'носитель (носитель)',
 '" («)',
 'союз-2.1а (Союз-2.1а)',
 '" (»)',
 'с (с)',
 '11 (11)',
 'спутник (спутниками)',
 'стартовать (стартовала)',
 'с (с)',
 'космодром (космодрома)',
 'восточный (Восточный)',
 '. (.)']
['Об (Об)',
 'это (этом)',
 'сообщать (сообщает)',
 'космический (Космический)',
 'центр (центр)',
 '" («)',
 'южный (Южный)',
 '" (»)',
 ', (,)',
 'который (который)',
 'вести (ведет)',
 'прямой (прямую)',
 'трансляция (трансляцию)',
 'запуск (запуска)',
 '. (.)']


Вместе со Spacy ставится библиотека dasplaycy, которая умеет отображать текст в виде деревьев зависимости. Посмотрим на первые два предложения.

In [34]:
for s in list(doc.sents)[:2]:
    displacy.render(s, style="dep", minify=True, jupyter=True, options={"distance":90})

Для предложения определено свойство `root`, которое является корневой вершиной предложения.  
Помимо леммы и токена, каждая вершина хранит список потомков `children`, список родителей `ancestors`, вид зависимости (роль слова при родителе) `dep_`, часть речи `tag_`.

In [35]:
sent0 = list(doc.sents)[0]
childs0 = list(sent0.root.children)
print(sent0.root, childs0)
print(childs0[0].lemma_, childs0[0].text, childs0[0].dep_, childs0[0].tag_, '->', list(childs0[0].ancestors)[0])

стартовала [Ракета, -, носитель, спутниками, космодрома, .]
ракета Ракета nsubj NOUN -> стартовала


При анализе текстов существует задача оценки тональности текста. Тональность текста определяет какого роде слова содержатся в тексте - негативные или позитивные. Например, если у нас имеется отзыв на какой-то товар, услугу или организацию, мы можем определить, положительный это отзыв или отрицательный. Для оценки тональности отзыва используют, среди прочего, словари оценочной лексики.

Один отзыв может содержать как критику, так и похвалу, в зависимости от того, что в нем рассматривается (Прекрасное обслуживание, но пироженые были несвежими). Код ниже использует словарь оценочной лексики для того, чтобы понять, какие глаголы сочетаются чаще с отрицательно окрашенными словами, а какие - с положительными.

In [36]:
# Загружаем словарь оценочной лексики.
with open("data/rusentilex_2017.txt", encoding="utf-8") as senti_file: 
    for i in range(18): # Пропускаем заголовок файла.
        _ = senti_file.readline()
        
    senti_words = {line.split(", ")[0]: line.split(", ")[3] for line in senti_file.readlines()}

In [37]:
# Посмотрим на первые 10 слов словаря.
list(senti_words.items())[:10]

[('аборт', 'negative'),
 ('абортивный', 'negative'),
 ('абракадабра', 'negative'),
 ('абсурд', 'negative'),
 ('абсурдность', 'negative'),
 ('абсурдный', 'negative'),
 ('авантюра', 'negative'),
 ('авантюризм', 'negative'),
 ('авантюрист', 'negative'),
 ('авантюристический', 'negative')]

In [38]:
# Поиск эмоционально окрашенных потомков для SpaCy.
def find_sentiments_spacy(node, senti_words, sentiments):
    for child in node.children:
        if child.lemma_ in senti_words.keys():
            if node.lemma_ not in sentiments.keys():
                sentiments[node.lemma_] = defaultdict(int)
            sentiments[node.lemma_][senti_words[child.lemma_]] += 1


In [39]:
# Перебираем все прдложения первой новости и ищем эмоционально окрашенные слова.
sentiments = {}
for sent in doc.sents:
    find_sentiments_spacy(sent.root, senti_words, sentiments)
sentiments

{'объяснить': defaultdict(int, {'negative': 2}),
 'заявить': defaultdict(int, {'negative': 1})}

In [40]:
# Найдем все эмоционально окрашенные фразы во всех новостях.
sentiments = {}
for sent_text in tqdm(text_news):
    doc = nlp(sent_text)
    for sentence in doc.sents:
        find_sentiments_spacy(sentence.root, senti_words, sentiments)


100%|███████████████████████████████████████| 1708/1708 [01:38<00:00, 17.26it/s]


In [41]:
[(senti, vals) for senti,vals in sentiments.items() if vals['negative']>30], '\n----\n',\
[(senti, vals) for senti,vals in sentiments.items() if vals['positive']>30]

([('признать',
   defaultdict(int, {'negative': 44, 'positive': 7, 'neutral': 2})),
  ('стать',
   defaultdict(int,
               {'negative': 42,
                'positive': 35,
                'neutral': 18,
                'positive/negative': 3})),
  ('мочь',
   defaultdict(int,
               {'negative': 34,
                'positive': 18,
                'neutral': 11,
                'positive/negative': 3,
                'пострадать': 1})),
  ('произойти',
   defaultdict(int,
               {'negative': 58,
                'positive/negative': 2,
                'neutral': 5,
                'positive': 4}))],
 '\n----\n',
 [('принять', defaultdict(int, {'positive': 39, 'neutral': 4, 'negative': 5})),
  ('обратить', defaultdict(int, {'positive': 64, 'negative': 2})),
  ('стать',
   defaultdict(int,
               {'negative': 42,
                'positive': 35,
                'neutral': 18,
                'positive/negative': 3}))])

## Синтаксический анализатор Stanza

Одним из синтаксических анализаторов является [Stanza](https://stanfordnlp.github.io/stanza/installation_usage.html).

In [42]:
import stanza

In [43]:
nlp2 = stanza.Pipeline('ru', use_gpu=False)

2023-05-12 09:02:32 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-05-12 09:02:34 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |
| lemma     | syntagrus |
| depparse  | syntagrus |
| ner       | wikiner   |

2023-05-12 09:02:34 INFO: Use device: cpu
2023-05-12 09:02:34 INFO: Loading: tokenize
2023-05-12 09:02:34 INFO: Loading: pos
2023-05-12 09:02:35 INFO: Loading: lemma
2023-05-12 09:02:35 INFO: Loading: depparse
2023-05-12 09:02:35 INFO: Loading: ner
2023-05-12 09:02:37 INFO: Done loading processors!


In [44]:
doc = nlp2(text_news[11])


In [45]:
doc.sentences[0].words

[{
   "id": 1,
   "text": "Ракета",
   "lemma": "ракета",
   "upos": "NOUN",
   "feats": "Animacy=Inan|Case=Nom|Gender=Fem|Number=Sing",
   "head": 12,
   "deprel": "nsubj",
   "start_char": 0,
   "end_char": 6
 },
 {
   "id": 2,
   "text": "-",
   "lemma": "-",
   "upos": "PUNCT",
   "head": 3,
   "deprel": "punct",
   "start_char": 6,
   "end_char": 7
 },
 {
   "id": 3,
   "text": "носитель",
   "lemma": "носитель",
   "upos": "NOUN",
   "feats": "Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing",
   "head": 1,
   "deprel": "appos",
   "start_char": 7,
   "end_char": 15
 },
 {
   "id": 4,
   "text": "«",
   "lemma": "\"",
   "upos": "PUNCT",
   "head": 5,
   "deprel": "punct",
   "start_char": 16,
   "end_char": 17
 },
 {
   "id": 5,
   "text": "Союз",
   "lemma": "союз",
   "upos": "NOUN",
   "feats": "Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing",
   "head": 3,
   "deprel": "appos",
   "start_char": 17,
   "end_char": 21
 },
 {
   "id": 6,
   "text": "-",
   "lemma": "-",
   "upos": 

In [46]:
doc.sentences[0].words[0].lemma

'ракета'

In [47]:
doc.sentences[0].dependencies

[({
    "id": 12,
    "text": "стартовала",
    "lemma": "стартовать",
    "upos": "VERB",
    "feats": "Aspect=Perf|Gender=Fem|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act",
    "head": 0,
    "deprel": "root",
    "start_char": 44,
    "end_char": 54
  },
  'nsubj',
  {
    "id": 1,
    "text": "Ракета",
    "lemma": "ракета",
    "upos": "NOUN",
    "feats": "Animacy=Inan|Case=Nom|Gender=Fem|Number=Sing",
    "head": 12,
    "deprel": "nsubj",
    "start_char": 0,
    "end_char": 6
  }),
 ({
    "id": 3,
    "text": "носитель",
    "lemma": "носитель",
    "upos": "NOUN",
    "feats": "Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing",
    "head": 1,
    "deprel": "appos",
    "start_char": 7,
    "end_char": 15
  },
  'punct',
  {
    "id": 2,
    "text": "-",
    "lemma": "-",
    "upos": "PUNCT",
    "head": 3,
    "deprel": "punct",
    "start_char": 6,
    "end_char": 7
  }),
 ({
    "id": 1,
    "text": "Ракета",
    "lemma": "ракета",
    "upos": "NOUN",
    "feats": "

In [48]:
# Поиск эмоционально окрашенных потомков для Stanza.
def find_sentiments_stanza(dependencies, senti_words, sentiments):
    for dep in dependencies:
        if dep[2].lemma in senti_words.keys():
            if dep[0].lemma not in sentiments.keys():
                sentiments[dep[0].lemma] = defaultdict(int)
            sentiments[dep[0].lemma][senti_words[dep[2].lemma]] += 1


In [49]:
sentiments = {}

for news1 in tqdm(text_news):
    doc = nlp2(news1)

    for sentence in doc.sentences:
        find_sentiments_stanza(sentence.dependencies, senti_words, sentiments)

100%|███████████████████████████████████████| 1708/1708 [46:38<00:00,  1.64s/it]


In [50]:
sentiments

{'комплект': defaultdict(int, {'neutral': 1}),
 'провести': defaultdict(int, {'neutral': 20, 'negative': 3, 'positive': 4}),
 'являться': defaultdict(int,
             {'neutral': 8,
              'positive': 18,
              'negative': 24,
              'positive/negative': 2}),
 'в': defaultdict(int, {'neutral': 34, 'positive': 9, 'negative': 4}),
 'головка': defaultdict(int, {'positive': 1}),
 'отсутствовать': defaultdict(int, {'positive': 2}),
 'порядок': defaultdict(int, {'positive': 1, 'neutral': 3, 'negative': 2}),
 'аппаратура': defaultdict(int, {'positive': 1}),
 'уделяться': defaultdict(int, {'positive': 2}),
 'Совет': defaultdict(int, {'positive': 11}),
 'способный': defaultdict(int,
             {'positive/negative': 4, 'negative': 6, 'neutral': 2}),
 'сдерживание': defaultdict(int, {'negative': 3, 'positive': 1}),
 'выразить': defaultdict(int, {'positive': 13, 'negative': 29, 'neutral': 1}),
 None: defaultdict(int,
             {'positive': 689,
              'negative':

In [51]:
[(senti, vals) for senti,vals in sentiments.items() if vals['negative']>10], '\n----\n',\
[(senti, vals) for senti,vals in sentiments.items() if vals['positive']>10]

([('являться',
   defaultdict(int,
               {'neutral': 8,
                'positive': 18,
                'negative': 24,
                'positive/negative': 2})),
  ('выразить',
   defaultdict(int, {'positive': 13, 'negative': 29, 'neutral': 1})),
  (None,
   defaultdict(int,
               {'positive': 689,
                'negative': 757,
                'neutral': 117,
                'positive/negative': 86})),
  ('результат',
   defaultdict(int,
               {'negative': 29,
                'neutral': 9,
                'positive': 21,
                'positive/negative': 3})),
  ('получить',
   defaultdict(int,
               {'negative': 60,
                'positive': 31,
                'neutral': 6,
                'positive/negative': 4})),
  ('страдать',
   defaultdict(int,
               {'negative': 30,
                'neutral': 4,
                'positive/negative': 2,
                'positive': 0})),
  ('один', defaultdict(int, {'neutral': 6, 'negative': 2